In [22]:
import pandas as pd
import json
import sys
import re
from datetime import datetime
from collections import defaultdict
from unidecode import unidecode
from string import ascii_lowercase
import sqlalchemy as sa

In [75]:
class TableCollector(object):
    
    def __init__(self, driver='pyodbc'):
        
        self.DRIVER_NAME = driver
        
        assert self.DRIVER_NAME in ['pyodbc', 'pymssql'], "wrong driver name - choose pyodbc or pymssql"
        
        try:
            self.SERVER, self.USER, self.PORT, self.PWD, self.DRIVER, self.DB_NAME = \
            [line.split("=")[-1].strip() for line in open("config/connection.ini", 'r').readlines() if line.strip()]          
        except:
            print("problem with configuration file, exiting..")
            sys.exit(0)
            
    def connect(self):
        
        self.CONNSTR = 'mssql+{}://{}:{}@{}:{}/{}'.format(self.DRIVER_NAME, self.USER, self.PWD, 
                                                          self.SERVER, self.PORT, self.DB_NAME)
        
        if self.DRIVER_NAME == 'pyodbc':
            import pyodbc
            self.CONNSTR += '?driver=' + self.DRIVER
        elif self.DRIVER_NAME == 'pymssql':
            import pymssql
            
        eng = sa.create_engine(self.CONNSTR)
        self.CONNECTION = eng.connect()
        
        return self
    
    def get_customers(self):
        
        self.TODAY = datetime.now().strftime("%Y%m%d")
        pick_customers_qry = """
                SELECT [CustomerID] as [cust_id],
                RTRIM(LTRIM(LOWER(ISNULL([FirstName],'')))) + ' ' +
                RTRIM(LTRIM(LOWER(ISNULL([MiddleName],'')))) + ' ' +
                RTRIM(LTRIM(LOWER(ISNULL([LastName],'')))) as [full_name]
                FROM [DWSales].[dbo].[tbl_LotusCustomer]
                where ([CustomerListID] = 2) and ([ModifiedDate] = '""" + self.TODAY + "')"
        
        self.TODAYS_CUSTOMERS = pd.read_sql(pick_customers_qry, self.CONNECTION, dtype=str)
        
        return self
    

In [106]:
class EthnicityDetector(object):
    
    def __init__(self, df, ethnicity_list):
        
        self.DATA_DIR = "/Users/ik/Data/"
        self.NAME_DATA_DIR = self.DATA_DIR + "names/"
        self.ethnicity_list = ethnicity_list
        self.input_df = df
        print(self.input_df.head())
        
        # load name and surname databases
        self.name_dict = json.load(open(self.NAME_DATA_DIR + "names_26092017.json", "r"))
        self.surname_dict = json.load(open(self.NAME_DATA_DIR + "surnames_26092017.json", "r"))
        # make name and surname dictionaries by letter for required ethnicities
        self.names = defaultdict(lambda: defaultdict(set))
        self.surnames = defaultdict(lambda: defaultdict(set))
    
    def _create_ethnic_dicts(self):
        
        for ethnicity in self.ethnicity_list:
            
            if ethnicity in self.name_dict:
                self.names[ethnicity] = {letter: {w["name"] for w in self.name_dict[ethnicity] 
                                                 if w["name"][0] == letter} for letter in ascii_lowercase}
            else:
                self.names[ethnicity] = {}
                
            if ethnicity in self.surname_dict:
                self.surnames[ethnicity] = {letter: {w for w in self.surname_dict[ethnicity] 
                                                 if w[0] == letter} for letter in ascii_lowercase}
            else:
                self.surnames[ethnicity] = {}
                
        return self
    
    def _clean_input(self):
        
        # replace separators with white spaces, then make sure there's only 1 white space separating name parts
        self.input_df["full_name"] = (self.input_df["full_name"]
                                                                .str.replace(re.compile(r"[-'_]"), " ")
                                                                  .str.split().str.join(' ').str.strip())
        # only keep names that have parts consisting of letters and longer than a single character
        self.input_df = self.input_df.loc[self.input_df["full_name"].apply(lambda _: " ".join([p for p in _.split() if ((len(p) > 1) and p.isalpha())])).notnull(), :]
        
        return self
    
    def _find_in_name(self):
   
        self.input_df["name_ethn"] = self.input_df["full_name"].apply(lambda _: "|".join([ethnicity for ethnicity in self.ethnicity_list 
                                                                      if _ in self.names[ethnicity][_[0]]]))
            
        return self
        
        
        

In [107]:
df = pd.read_csv("/Users/ik/Data/temp/sample_new_customer_names.csv")
ed = EthnicityDetector(df, ["indian", "filipino"])

   cust_id          full_name
0    12374     steve  balzary
1    13763  annette  matheson
2    52404   andrew g wiseman
3    53306     bruce  fenwick
4    54140  lionel  werbeloff


In [108]:
ed._create_ethnic_dicts()._clean_input()
ed.input_df.head(40)

,cust_id,full_name
0,12374,steve balzary
1,13763,annette matheson
2,52404,andrew g wiseman
3,53306,bruce fenwick
4,54140,lionel werbeloff
5,55702,prudence leslie
6,55869,annmaree hooker
7,56700,mike bollen
8,56711,sylvia toop
9,57802,paula t mackinlay
